# Assignment 02: K-Means Clustering

I want you to implement a function to calculate the k-means clustering of a dataframe of $m$ observations with $n$ attributes.

Your implementation function should have the signature:
    
    cluster_kmeans(df, k)
    
where
    
    df is a Pandas dataframe of m observations with n attributes; m rows with n columns (excluding index;
    the index will contain the label for each observation and is not considered an attribute)
    
    k is the number of clusters to find

The function should return a new dataframe with a single column: the cluster label for each observation.
(Copy the index from the input dataframe into the output dataframe to keep them consistent.).  It should also return the last Sum-of-the-Square-Errors (SSE) from the clustering.

For the proximity measure, you can use Euclidean distance as the metric.  Sum-of-the-Square-Errors is the objective function.  You can assume that the attributes have been standardized (i.e. properly-transformed) prior to k-means being called.

**Keep your final solution notebook tidy.  I will be testing your code by first executing every cell in the notebook and then calling your `cluster_kmeans` with my test data.  If a cell cause the interpreter to throw an exception, I will not be able to test the notebook.**

You may write additional functions and tests in this notebook to write and test your solution. *I should be able to add a cell at the end of this notebook and have it test your function by using  the kernel's "Restart and Run All" feature.*

Start by creating test dataframes where the clusters should be obvious.  Then test the end of the process by being able to plot a dataframe's attributes on a 2D plot with the correct cluster label.  (Since these are test points, you know what that label should be.)  Once you know you can generate test data and plot the solution, begin working on the cluster_kmean's function.  

After playing with a small number of points in test dataframes to check your solution, you may consider using [distribution sampling functions from numpy](https://docs.scipy.org/doc/numpy-1.14.0/reference/routines.random.html) to generate points in a cluster for testing.  (Combine the clusters you generate into a single dataframe to test how well your algorithm works.)

Do note that the labels from each call to cluster_kmeans may change: that is cluster 1 and cluster 2 might appear as cluster 2 and cluster 1.  Since you know the original labels from your test data as you generated it yourself, you should be able to figure out which belongs to which original label during testing.

**<span style="color:purple">The project will be due after Spring break on Wednesday, March 11th @ 10PM via Mimir Classroom.</span>**



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math  

In [2]:
from sklearn import preprocessing

In [1]:
%matplotlib inline
def cluster_kmeans(df, k):
    """
    Clusters the m observations of n attributes 
    in the Pandas' dataframe df into k clusters.
    
    Euclidean distance is used as the proximity metric.
    
    Arguments:
        df   pandas dataframe of m rows with n columns (excluding index)
        k    the number of clusters to search for
        
    Returns:
        a m x 1 dataframe of cluster labels for each of the m observations
        retaining the original dataframe's (df's) index
        
        the final Sum-of-Error-Squared (SSE) from the clustering
    """
    # Sample fron the original df
    sample_df=df.sample(n = k)
    obs, attr= df.shape
    # Make copies 
    copy_df=df.copy()
    flag=0
    sse_old=0
    while (flag==0): 
        sse=0
        Labels=[]
        for i in range(0, obs):
            dist= []
            for j in range(0,k):
                #Calculate Eucledian distance
                diff=list((df.iloc[i,:]-sample_df.iloc[j,:])**2)
                eu_dist=(sum(diff))**(1/attr)
                dist.append(eu_dist) 
            #Add Labels to the observations based on the variable they are close to
            label=(dist.index(min(dist)))
            Labels.append(label)
            # Calculate SSE
            sse=sse+((min(dist) )**2)
        sse=sse**(1/2)
        copy_df['Labels']=Labels
        # Stopping criteria is change in SSE should be 2 %
        if (sse_old !=0):
            if(abs(sse_old-sse)/sse_old<=0.02):
                flag=1 
                return (copy_df['Labels'].to_frame(), sse)
            else:
                sse_old=sse
                sample_df.drop(sample_df.index, inplace=True)
                # Now pick random values from each label and add it to the sample df
                for val in range(0,k):
                    sample_df = pd.concat([sample_df, copy_df[copy_df['Labels']==val].iloc[:,0:attr].sample(n=1)])
        else:
            sse_old=sse
            sample_df.drop(sample_df.index, inplace=True)
            for val in range(0,k):
                sample_df = pd.concat([sample_df, copy_df[copy_df['Labels']==val].iloc[:,0:attr].sample(n=1)])
        #copy_df.plot.scatter(copy_df.columns[0],copy_df.columns[1], c='Labels' ,colormap='viridis')